## Objective

Matrix Factorization이란, 주어진 고객과 제품 간의 상호작용 행렬로부터 고객의 취향 정보 행렬과 제품의 특성 정보 행렬을 도출하는 과정을 의미합니다. 

<img src="https://i.imgur.com/QnC5xQx.png" width="800" >
<br>
위의 그림을 다르게 묘사하면 아래와 같습니다.
<br>
<img src="https://i.imgur.com/zvx2JNs.png" width="400" >

유저의 취향 행렬과 제품의 특성 행렬을 얻게 되면, 우리는 크게 3가지의 작업을 할 수 있게 됩니다.<br>
- Rating Prediction : 유저가 경험해보지 않은 아이템에 대한 선호도 예측하기
- User Clustering : 유사한 취향을 가진 유저 묶기 
- Item Clustering : 유사한 특성을 가진 아이템 묶기

Matrix Factorization의 핵심은 어떻게 User Matrix와 Item Matrix을 구할 수 있는가입니다. 


고객행동 데이터 중 암묵 데이터를 활용할 때 사용하는 Matrix Factorization 알고리즘 중 하나인 베이지안 개인화 랭킹 알고리즘(BPR, Bayesian Personalized Ranking)을 Tensorflow을 통해 구현해보도록 하겠습니다.

### 필요 모듈 가져오기

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
tqdm.pandas()
np.set_printoptions(5,)

### 데이터 가져오기 

In [2]:
from tensorflow.keras.utils import get_file

ROOT_URL = "https://craftsangjae.s3.ap-northeast-2.amazonaws.com/data/"

# 데이터 가져오기
play_path = get_file("lastfm_play.csv",
                     ROOT_URL+"lastfm_play.csv")
artist_path = get_file("lastfm_artist.csv",
                       ROOT_URL+"lastfm_artist.csv")
user_path = get_file("lastfm_user.csv",
                     ROOT_URL+"lastfm_user.csv")

play_df = pd.read_csv(play_path)
artist_df = pd.read_csv(artist_path)
user_df = pd.read_csv(user_path)

## Bayesian Personalized Ranking 구현하기

### Input 구성하기

Bayesian Personalized Ranking의 핵심 아이디어는 바로 

> 고객이 구매/청취한 제품은 고객이 구매/청취하지 않은 제품보다 선호도가 높다

입니다. Bayesain Personalized Ranking에서는 고객과 고객이 청취한 아티스트, 그리고 고객이 청취하지 않은 아티스트 총 3 개의 입력이 들어가게 됩니다.

In [3]:
from tensorflow.keras.layers import Input

user_id = Input(shape=(), name='user')  # name : user
pos_item_id = Input(shape=(), name='pos_item') # name : positive_item 
neg_item_id = Input(shape=(), name='neg_item') # name : negative_item

### 임베딩 레이어 구성하기

Bayesian Personalized Ranking에서 해야하는 것은 상호작용 정보를 통해 유저와 아이템에 대한 적절한 임베딩 값을 추론하는 것입니다. 임베딩 레이어를 아래처럼 생성합니다. 이 때 Item Embedding의 경우, 각 아이템 별 편향 정보(Bias)를 추가하기 위해 Num Factor에 1을 더했습니다.

In [4]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import RandomUniform

num_user = play_df.user_id.max() + 1
num_item = play_df.artist_id.max() + 1
num_factor = 32

# 초기화 함수 : Uniform 분포
init_range = 1 / (2*num_factor)
initializer = RandomUniform(minval=-init_range, maxval=init_range)

# Embedding Layer 선언하기
user_embedding_layer = Embedding(num_user, num_factor, 
                                 embeddings_initializer=initializer,
                                 name='user_embedding')
item_embedding_layer = Embedding(num_item, num_factor, 
                                 embeddings_initializer=initializer,
                                 name='item_embedding')
item_bias_layer = Embedding(num_item, 1, 
                            embeddings_initializer='zeros',
                            name='item_bias')

### Item Embedding, User Embedding 구하기

Tensorflow Keras에서 Item Embedding의 값과 User Embedding의 값을 가져오는 것은 매우 간단합니다. 우리는 층의 연결을 통해 가져올 수 있습니다.

#### Item Embedding 구하기

이 때 주의해야 하는 것은 positive item과 negative item은 같은 임베딩 레이어에서 가져와야 합니다. <br>

In [5]:
pos_item_embedding = item_embedding_layer(pos_item_id)
neg_item_embedding = item_embedding_layer(neg_item_id)

#### User Embedding 구하기

In [6]:
from tensorflow.keras.layers import Concatenate

user_embedding = user_embedding_layer(user_id)

#### Item Bias 구하기

In [7]:
pos_item_bias = item_bias_layer(pos_item_id)
neg_item_bias = item_bias_layer(neg_item_id)

### Score 계산하기

우리는 고객이 본 아이템에 대한 Score와 고객이 보지 않은 아이템에 대한 Score의 차이가 극대화되도록 학습하게 됩니다.

In [8]:
from tensorflow.keras.layers import Dot
from tensorflow.keras.layers import Subtract

pos_score = (
    Dot(axes=(1,1))([user_embedding, pos_item_embedding]) + pos_item_bias)
neg_score = (
    Dot(axes=(1,1))([user_embedding, neg_item_embedding]) + neg_item_bias)

score = Subtract()([pos_score, neg_score])

### Regularization 적용하기


Matrix Factoriation은 쉽게 과적합이 발생합니다. 특히 `item_embedding`의 bias factor는 쉽게 과적합될 수 있습니다. 이를 줄이기 위해 regularization으로 `l2`를 두게 됩니다.

In [9]:
from tensorflow.keras.regularizers import l2

l2_reg = 1e-2

l2_pos_item = tf.reduce_sum(pos_item_embedding**2)
l2_neg_item = tf.reduce_sum(neg_item_embedding**2)
l2_user = tf.reduce_sum(user_embedding**2)

weight_decay = l2_reg * (l2_pos_item+l2_neg_item+l2_user)

### Model 구성하기

입력값은 크게 세가지 `user_id`, `pos_item_id`,`neg_item_id`으로 나뉘어집니다. 그리고 출력값은 보지 않은 아이템에 대한 선호도보다 본 아이템에 대한 선호도가 높을 확률(`probs`)이 됩니다.

In [16]:
def create_bpr_model(num_user, num_item, num_factor, l2_reg=1e-2):
    user_id = Input(shape=(), name='user')
    pos_item_id = Input(shape=(), name='pos_item')
    neg_item_id = Input(shape=(), name='neg_item')
    
    initializer = RandomUniform(minval=-1/num_factor, maxval=1/num_factor)
    
    user_embedding_layer = Embedding(num_user, num_factor, 
                                     embeddings_initializer=initializer,
                                     name='user_embedding')
    item_embedding_layer = Embedding(num_item, num_factor, 
                                     embeddings_initializer=initializer,
                                     name='item_embedding')
    item_bias_layer = Embedding(num_item, 1, 
                                embeddings_initializer='zeros',
                                name='item_bias')
    
    pos_item_embedding = item_embedding_layer(pos_item_id)
    neg_item_embedding = item_embedding_layer(neg_item_id)
    
    user_embedding = user_embedding_layer(user_id)
    
    pos_item_bias = item_bias_layer(pos_item_id)
    neg_item_bias = item_bias_layer(neg_item_id)
        
    pos_score = (
        Dot(axes=(1,1))([user_embedding, pos_item_embedding]) + pos_item_bias)
    neg_score = (
        Dot(axes=(1,1))([user_embedding, neg_item_embedding]) + neg_item_bias)

    score = Subtract()([pos_score, neg_score])
    
    model = Model([user_id, pos_item_id, neg_item_id], score)
    
    l2_pos_item = tf.reduce_sum(pos_item_embedding**2)
    l2_neg_item = tf.reduce_sum(neg_item_embedding**2)
    l2_user = tf.reduce_sum(user_embedding**2)

    weight_decay = l2_reg * (l2_pos_item+l2_neg_item+l2_user)

    model.add_loss(weight_decay)
    return model

In [17]:
from tensorflow.keras.models import Model

model = create_bpr_model(num_user, num_item, 32, 1e-2)

### Model 컴파일하기

옵티마이저로는 `Adagrad`를 이용합니다. Sparse한 input을 다룰 때에 `Adagrad`는 SGD보다 훨씬 빠르게 수렴시킬 수 있습니다.

그리고 Loss를 구할 때, aggregation 방식은 **평균** 대신 **합계**가 씁니다. 모든 input Feature에 대해 가중치를 공유하는 `Conv`,`Dense`,`RNN` 등과 달리, 각 input feature에 대해서 독립적으로 가중치를 적용하기 때문에 평균으로 할 경우 각 임베딩 weight에 대한 가중치가 배치 사이즈만큼 나누어주는 효과가 발생하기 때문입니다.

In [18]:
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.metrics import BinaryAccuracy

def bpr_loss(y_true, y_pred):
    return tf.reduce_sum(-tf.math.log(sigmoid(y_pred)))

model.compile(Adagrad(1e-1), loss=bpr_loss,
              metrics=[bpr_loss, BinaryAccuracy(threshold=0)])

### 학습 데이터 파이프라인 구성하기

BPR의 손실함수인 Triplet Loss를 계산하기 위해서는 Dataset을 Bootstraping을 통해 샘플링해야합니다. 이 때 주의해야 할 것 중 하나는 바로 negative Case를 Sampling할 때에도 원래 item의 분포와 동일하게 sampling을 해주어야 합니다. 

In [19]:
batch_size = 4096

bootstrap = play_df.sample(frac=1., replace=True)
user_ids = bootstrap.user_id.values
pos_item_ids = bootstrap.artist_id.values
neg_item_ids = play_df.artist_id.sample(frac=1., replace=True).values

X = {
    "user": user_ids,
    "pos_item": pos_item_ids,
    "neg_item": neg_item_ids
}
dummy_y = np.ones((len(bootstrap), 1))

dataset = (
    tf.data.Dataset
    .from_tensor_slices((X,dummy_y))
    .batch(batch_size)) # 배치 단위로 record 묶기

In [20]:
def bootstrap_dataset(df, batch_size=4096):
    bootstrap = df.sample(frac=1., replace=True)
    user_ids = bootstrap.user_id.values
    pos_item_ids = bootstrap.artist_id.values
    neg_item_ids = df.artist_id.sample(frac=1., replace=True).values

    X = {
        "user": user_ids,
        "pos_item": pos_item_ids,
        "neg_item": neg_item_ids
    }
    dummy_y = np.ones((len(bootstrap), 1))
    
    dataset = (
        tf.data.Dataset
        .from_tensor_slices((X,dummy_y))
        .batch(batch_size)) # 배치 단위로 record 묶기
    
    return dataset

### 모델 학습하기

epoch 10번에 걸쳐 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 pair를 생성하도록 하였습니다. 

In [21]:
num_epoch = 10
batch_size = 4096

for i in range(num_epoch):
    print(f"{i+1}th epoch")
    dataset = bootstrap_dataset(play_df, batch_size)
    model.fit(dataset)

1th epoch
4225/4225 [==============================] - 81s 19ms/step - loss: 2271.9033 - bpr_loss: 2099.7532 - binary_accuracy: 0.7349
2th epoch
4225/4225 [==============================] - 70s 16ms/step - loss: 1569.7872 - bpr_loss: 1228.6342 - binary_accuracy: 0.8907
3th epoch
4225/4225 [==============================] - 68s 16ms/step - loss: 1437.0178 - bpr_loss: 1060.0135 - binary_accuracy: 0.9085
4th epoch
4225/4225 [==============================] - 68s 16ms/step - loss: 1378.5002 - bpr_loss: 980.3517 - binary_accuracy: 0.9170
5th epoch
4225/4225 [==============================] - 68s 16ms/step - loss: 1344.8147 - bpr_loss: 931.5139 - binary_accuracy: 0.9220
6th epoch
4225/4225 [==============================] - 69s 16ms/step - loss: 1322.5952 - bpr_loss: 897.7891 - binary_accuracy: 0.9255
7th epoch
4225/4225 [==============================] - 65s 15ms/step - loss: 1306.3130 - bpr_loss: 872.2475 - binary_accuracy: 0.9284
8th epoch
4225/4225 [==============================] - 82s 

## 학습된 임베딩 행렬 가져오기
학습된 임베딩 행렬은 `model.user_factors`와 `model.item_factors`에 저장되어 있습니다.

In [22]:
user_embeddings = model.get_layer('user_embedding').get_weights()[0]

item_embeddings = model.get_layer('item_embedding').get_weights()[0]

item_bias = model.get_layer('item_bias').get_weights()[0]

좀 더 가독성을 높이기 위해서 각 임베딩 행 별로 아티스트의 이름과 유저의 id를 매칭시켜 데이터프레임(dataframe)을 구성하도록 하겠습니다.

In [39]:
item_bias

array([[ 0.16628],
       [ 0.17658],
       [-0.02754],
       ...,
       [ 0.10557],
       [ 0.07174],
       [ 0.16388]], dtype=float32)

In [40]:
user_embedding_df = pd.DataFrame(user_embeddings,
                                 index=user_df.user_id)

artist_embedding_df = pd.DataFrame(item_embeddings,
                                   index=artist_df.artist_name)

artist_embedding_df[num_factor] = item_bias[:,0]
artist_embedding_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
artist_name,,,,,,,,,,,,,,,,,,,,,
betty blowtorch,0.347647,0.345300,0.283413,0.124686,0.457707,-0.814453,-0.275293,-0.316983,0.060874,-0.383601,...,0.130088,0.014209,0.192937,-0.425993,-0.030325,0.125157,-0.189928,0.053410,-0.054539,0.166278
die Ärzte,-0.609451,-0.122681,0.116881,-0.022823,0.063232,0.109007,0.188018,0.111158,-0.349701,0.200430,...,-0.153900,-0.195210,-0.152610,-0.310218,-0.121051,-0.247437,-0.001620,0.138030,0.167068,0.176579
melissa etheridge,0.183831,0.231989,0.406555,0.037937,-0.058550,-0.229385,-0.528049,0.191282,0.161729,0.166509,...,-0.135420,-0.035317,-0.147862,0.521583,0.639455,-0.345175,-0.664175,-0.121457,0.261626,-0.027541
elvenking,-0.558999,-0.287427,0.143505,0.405830,0.470800,0.099769,0.181633,-0.412667,-0.172376,-0.581064,...,-0.056614,-0.340192,0.151076,-0.496645,0.362071,-0.291823,-0.117428,-0.229639,0.362357,-0.488120
juliette & the licks,-0.127612,0.213415,0.042106,-0.131089,-0.251563,-0.844970,-0.409262,-0.352434,-0.104454,0.114784,...,0.250796,0.141587,-0.242373,-0.092049,0.015458,0.240175,0.226212,0.686269,0.137652,0.663241


예를 들면 가수 리한나의 임베딩 결과는 아래와 같습니다.

In [41]:
print(artist_embedding_df.loc["rihanna"].values)

[-0.26963  0.04856  0.33164  0.24663 -0.27721  0.20203 -0.0955   0.35793
 -0.25274  0.34072  0.31541 -0.18788 -0.3396  -0.41544  0.11444  0.3265
  0.2011  -0.15    -0.38465  0.55826 -0.13538 -0.37079 -0.30983 -0.03116
  0.46811  0.14085  0.32425  0.08879 -0.20715  0.0675  -0.09464  0.48116
  0.14322]


###  즐겨듣는 아티스트와 유사한 아티스트 추천하기

내적 연산(dot product)를 통해 두 벡터의 유사도를 계산할 수 있다고 배웠습니다. 각 아티스트의 임베딩을 다른 아티스트의 임베딩과 내적 연산하여 아티스트 사이의 유사도를 구하면, 각 아티스트와 가장 높은 유사도를 가진 아티스트를 찾을 수 있습니다.

#### 제이슨 므라즈와 유사한 아티스트 찾기

In [42]:
target_embedding = artist_embedding_df.loc['jason mraz']

(
    artist_embedding_df
    .dot(target_embedding)
    .sort_values(ascending=False)
    .iloc[:10]
)

artist_name
mozella                   2.814972
josh kelley               2.806064
matt wertz                2.772019
jason reeves              2.766373
teddy geiger              2.703305
justin nozuka             2.701589
jeremy kay                2.651573
jamie scott & the town    2.649342
eric hutchinson           2.634584
gavin degraw              2.634161
dtype: float32

#### 브리트니 스피어스와 유사한 아티스트 찾기

In [43]:
target_embedding = artist_embedding_df.loc['britney spears']

(
    artist_embedding_df
    .dot(target_embedding)
    .sort_values(ascending=False)
    .iloc[:10]
)

artist_name
girlicious          3.007082
the saturdays       2.964588
billie              2.933447
paris hilton        2.902036
victoria beckham    2.900184
nadia oh            2.896300
kate alexa          2.894922
agnes carlsson      2.880454
basim               2.862728
alesha dixon        2.859251
dtype: float32

#### 에미넴과 유사한 아티스트 찾기

In [44]:
target_embedding = artist_embedding_df.loc['eminem']

(
    artist_embedding_df
    .dot(target_embedding)
    .sort_values(ascending=False)
    .iloc[:10]
)

artist_name
lil jon                      2.426599
eminem & 50 cent             2.420105
afroman                      2.416948
d12                          2.393669
eminem                       2.388535
silkk the shocker            2.377112
diaz                         2.366818
herreløse                    2.342267
50 cent                      2.319996
jamie kennedy & stu stone    2.305045
dtype: float32

### 자신의 취향과 비슷한 특성을 가진 아티스트 추천하기

앞서 한 아티스트의 임베딩 벡터와 전체 아티스트의 임베딩 행렬을 이용해 아티스트 사이의 유사도를 구하여 유저가 즐겨듣는 아티스트와 유사한 아티스트를 추천해보았습니다. 이번에는 유저 임베딩 벡터와 전체 아티스트의 임베딩 행렬을 이용해 유저와 비슷한 아티스트를 찾아 추천해보겠습니다.

In [46]:
# 유저별 이미 들은 아티스트 리스트 구성하기
artistset_per_user = (
    play_df
    .groupby('user_id')
    ['artist_id']
    .apply(frozenset)
)

**메탈, 락과 같은 음악을 많은 들은 사람**

In [55]:
artist_id2name = dict(
    zip(artist_df.artist_id.values, artist_df.artist_name.values))

In [59]:
target_id = 300
target_user = user_embedding_df.loc[target_id]
target_user.loc[32] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['angelo badalamenti', 'red hot chili peppers', 'marilyn manson', 'led zeppelin', 'eric clapton', 'metallica', 'iron maiden', 'u2', 't.love', 'slipknot', 'queens of the stone age', "guns n' roses", 'iced earth', 'avril lavigne', 'guano apes', 'the offspring', 'alice in chains', 'in flames', 'pantera', 'john williams', 'daniel licht', 'high and mighty color', 'karmacoma', 'down', 'missile girl scoot', 'akira yamaoka', 'the kilimanjaro darkjazz ensemble', 'mondo generator', 'raging speedhorn', 'graeme revell', 'spiritual beggars', 'as i lay dying', 'frida snell', 'fatboy slim', 'pearl jam', 'isis', 'suicidal tendencies', 'black sabbath', 'stone sour', 'the smashing pumpkins', 'sigur rós', 'godsmack', 'pink', 'no doubt', 'nine inch nails']


target_user와 유사도가 높은 아티스트를 보면 아래와 같이 나옵니다.

In [60]:
(
    artist_embedding_df
    .dot(target_user) # target_user와 유사도 계산하기
    .sort_values(ascending=False)
    [:10]
)

artist_name
downface                 3.659676
deponeye                 3.342204
cad                      3.201091
vokee                    3.060442
jonathan davis           3.041707
gothacoustic ensemble    3.034197
jay gordon               3.024186
hurt                     3.017704
convergence              3.016421
godsmack                 3.004003
dtype: float64

**힙합을 많이 들은 사람**

In [61]:
target_id = 1200
target_user = user_embedding_df.loc[target_id]
target_user.loc[32] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['the stone roses', 'snow patrol', 'red hot chili peppers', 'nickelback', 'scouting for girls', 'blur', 'michael jackson', 'michael bublé', 'amy winehouse', 'u2', 'the fratellis', 'bob marley', 'busted', 'jay-z', 'prince', 'kanye west', 'jordin sparks', 'foo fighters', '2pac', 'arctic monkeys', '50 cent', 'coldplay', 'lil wayne', 'the jackson 5', 'shaggy', 'queen', 'the game', 'feeder', 'the streets', 'radiohead', 'jay-z and linkin park', 'rihanna', 'linkin park', 'nina simone', 'blink-182', 'basshunter', 'r. kelly', 'oasis', 'the view', 'usher', 'the verve', 'akon', 'rod stewart', 'stereophonics', 'hard-fi']


In [62]:
(
    artist_embedding_df
    .dot(target_user) # target_user와 유사도 계산하기
    .sort_values(ascending=False)
    [:10]
)

artist_name
danny bond                              3.428761
chris lee                               3.386015
4th25                                   3.378803
rush hour                               3.347842
multicyde                               3.251757
eamon                                   3.250701
dj greg j                               3.236316
jokeren                                 3.217016
justin timberlake & timbaland           3.212935
donavon frankenreiter & jack johnson    3.169665
dtype: float64

**댄스 음악을 좋아하는 사람**

In [65]:
target_id = 209
target_user = user_embedding_df.loc[target_id]
target_user.loc[32] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['gackt', 'christina aguilera', 'beyoncé', 'michael jackson', '安室奈美恵', '12012', '이효리', '新垣結衣', '久石譲', '鄭秀文', 'olivia ong', 'donawhale', '王力宏', 'enrique iglesias', 'alan', 'late night alumni', 'vanessa paradis', 'big bang', 'uverworld', 'abingdon boys school', 'britney spears', 'jennifer lopez', 'timbaland', 'ciara', 'bon jovi', 'avril lavigne', '中島美嘉', '浜崎あゆみ', 'm-flo', 'evanescence', '宇多田ヒカル', 'olivia', '倖田來未', 'spice girls', 'ashlee simpson', 'mariah carey', 'rihanna', 'linkin park', 'nelly furtado', 'madonna', 'enya', 'the pussycat dolls', 'kelly clarkson', 'michelle branch', 'frank sinatra', 'bee gees', 'justin timberlake', 'lady gaga', 'mink', 'boa', 'ガゼット', 'disney']


In [84]:
(
    artist_embedding_df
    .dot(target_user) # target_user와 유사도 계산하기
    .sort_values(ascending=False)
    [:10]
)

CPU times: user 202 ms, sys: 71 ms, total: 273 ms
Wall time: 112 ms


artist_name
sweetbox            4.096132
kate alexa          4.092253
koh  mr. saxman     4.075683
g-dragon            4.044978
kristine sa         4.032140
field of view       4.006198
tashannie           4.004965
โต๋-ศักดิ์สิทธิ์    3.957015
mateo               3.932843
阿桑                  3.928469
dtype: float64

## Matrix Factorization Serving

위와 같이 우리는 고객에 대한 Embedding Vector, 아티스트에 대한 Embedding Vector을 구했습니다. 그리고 이러한 Embedding Vector을 통해 어떻게 고객에게 적절히 추천할 수 있는지에 대해서도 알아봤습니다. 그럼 이를 통해 서비스를 한다면, 어떻게 해야 할까요?

![Imgur](https://i.imgur.com/aTkb7jB.png)


우리는 주기적으로 배치 서버를 통해 BPR 알고리즘을 통해 고객과 제품에 대한 Embedding Vector을 학습시켜 주면 됩니다. <br>
그럼 고객에게 추천을 할 때는 어떻게 해야할까요? 위와 같이 Pandas를 이용해, 유사도를 계산할 수 있지만 이럴 경우 우리는 대량의 요청이 들어왔을 때 빠르게 응답하기 어려울 수 있습니다. 고객의 경험을 위해서는 최소 100ms안에 요청을 처리할 수 있어야 합니다. 하지만 Pandas로는 어렵습니다. 이를 위해 Spotify에서는 자체 추천 서비스를 위한 도구인 Annoy를 오픈소스로 공개하였습니다.

### Spotify 팀의 서빙 도구, Annoy


Annoy는 위와 같이 전체 제품 중 고객에게 맞는 제품을 추천하기 위해, Dot 연산을 수행하여 제일 가까운 10개를 찾는 연산을 보다 빠르게 만들기 위한 라이브러리입니다. Annoy는 위와 같이 모든 제품과 Dot 연산을 수행하는 것이 아닌, 가장 가까울 것으로 예상되는 제품만을 추려 Dot 연산을 수행하는 방식으로 설계되어 있습니다. 즉 가까운 제품끼리 미리 클러스터링을 진행 후, 해당 제품끼리에서만 Dot 연산을 수행하여 최소의 연산으로 우리가 필요한 정보를 추출할 수 있도록 합니다.

In [67]:
from annoy import AnnoyIndex

## 1. 준비하기

우선 우리는 학습한 제품의 Embedding Vector을 Annoy에 저장해야 합니다.<br>

In [68]:
tree = AnnoyIndex(33, "dot") # Dot 유사도를 이용하기

for idx, value in enumerate(artist_embedding_df.values):
    tree.add_item(idx, value)

그리고 몇 개의 Tree를 통해 군집화를 할 것인지 설정합니다. <br> 이 때 Tree가 많을수록 좀 더 정확하게 군집화할 수 있고, 적을수록 좀 더 빠르게 연산을 수행할 수 있습니다.

In [69]:
tree.build(20)

True

## 2. Annoy를 통해 유사도 검색하기

**메탈, 락과 같은 음악을 많은 들은 사람**


In [74]:
target_id = 300
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['angelo badalamenti', 'red hot chili peppers', 'marilyn manson', 'led zeppelin', 'eric clapton', 'metallica', 'iron maiden', 'u2', 't.love', 'slipknot', 'queens of the stone age', "guns n' roses", 'iced earth', 'avril lavigne', 'guano apes', 'the offspring', 'alice in chains', 'in flames', 'pantera', 'john williams', 'daniel licht', 'high and mighty color', 'karmacoma', 'down', 'missile girl scoot', 'akira yamaoka', 'the kilimanjaro darkjazz ensemble', 'mondo generator', 'raging speedhorn', 'graeme revell', 'spiritual beggars', 'as i lay dying', 'frida snell', 'fatboy slim', 'pearl jam', 'isis', 'suicidal tendencies', 'black sabbath', 'stone sour', 'the smashing pumpkins', 'sigur rós', 'godsmack', 'pink', 'no doubt', 'nine inch nails']


Annoy를 통해 검색하려면 아래와 같이 작성하면 됩니다.

In [83]:
%%time
artist_indices = tree.get_nns_by_vector(target_user, 10)
artist_indices

CPU times: user 681 µs, sys: 103 µs, total: 784 µs
Wall time: 543 µs


Annoy를 통해 검색하게 되면 훨씬 더 빠른 속도 내에 처리할 수 있습니다. 현재 컴퓨터 성능으로는 대략 200배 정도 빨라집니다.

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같게 됩니다.

In [85]:
artist_embedding_df.index[artist_indices]

Index(['melody.', 'f4', 'twins', 'joanna wang', 'lmnt', '范逸臣', '林俊傑',
       'tanya chua', 'cẩm ly', '許茹芸'],
      dtype='object', name='artist_name')

**힙합을 많이 들은 사람**

In [86]:
target_id = 1200
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['the stone roses', 'snow patrol', 'red hot chili peppers', 'nickelback', 'scouting for girls', 'blur', 'michael jackson', 'michael bublé', 'amy winehouse', 'u2', 'the fratellis', 'bob marley', 'busted', 'jay-z', 'prince', 'kanye west', 'jordin sparks', 'foo fighters', '2pac', 'arctic monkeys', '50 cent', 'coldplay', 'lil wayne', 'the jackson 5', 'shaggy', 'queen', 'the game', 'feeder', 'the streets', 'radiohead', 'jay-z and linkin park', 'rihanna', 'linkin park', 'nina simone', 'blink-182', 'basshunter', 'r. kelly', 'oasis', 'the view', 'usher', 'the verve', 'akon', 'rod stewart', 'stereophonics', 'hard-fi']


Annoy를 통해 검색하면 아래와 같습니다.

In [87]:
artist_indices = tree.get_nns_by_vector(target_user, 10)

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같습니다.

In [88]:
artist_embedding_df.index[artist_indices]

Index(['chris lee', '4th25', 'dj greg j', 'pato', 'titofelix', 'karmacy',
       'scouting for girls', 'alain clark', 'jet', 'sir speedy'],
      dtype='object', name='artist_name')

**댄스 음악을 좋아하는 사람**

In [89]:
target_id = 209
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

['gackt', 'christina aguilera', 'beyoncé', 'michael jackson', '安室奈美恵', '12012', '이효리', '新垣結衣', '久石譲', '鄭秀文', 'olivia ong', 'donawhale', '王力宏', 'enrique iglesias', 'alan', 'late night alumni', 'vanessa paradis', 'big bang', 'uverworld', 'abingdon boys school', 'britney spears', 'jennifer lopez', 'timbaland', 'ciara', 'bon jovi', 'avril lavigne', '中島美嘉', '浜崎あゆみ', 'm-flo', 'evanescence', '宇多田ヒカル', 'olivia', '倖田來未', 'spice girls', 'ashlee simpson', 'mariah carey', 'rihanna', 'linkin park', 'nelly furtado', 'madonna', 'enya', 'the pussycat dolls', 'kelly clarkson', 'michelle branch', 'frank sinatra', 'bee gees', 'justin timberlake', 'lady gaga', 'mink', 'boa', 'ガゼット', 'disney']


Annoy를 통해 검색하면 아래와 같습니다.

In [90]:
artist_indices = tree.get_nns_by_vector(target_user, 10)

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같습니다.

In [91]:
artist_embedding_df.index[artist_indices]

Index(['melody.', 'f4', 'twins', 'joanna wang', 'lmnt', '范逸臣', '林俊傑',
       'tanya chua', 'cẩm ly', '許茹芸'],
      dtype='object', name='artist_name')

## 3. Annoy와 유사한 다양한 라이브러리들

Annoy와 같이 Embedding Vector의 유사도를 빠르게 계산해주는 라이브러리들이 많이 존재합니다. 이 중 Annoy는 성능은 준수한 수준에, 매우 간단하게 이용할 수 있어 많이 사용됩니다.

<img src="https://github.com/erikbern/ann-benchmarks/raw/master/results/glove-100-angular.png" width="500">